In [3]:
from random import randint, choice, shuffle, random
from pprint import pprint
from itertools import permutations
from math import inf as oo # Infinity (∞) is larger than any number
from math import sqrt
from time      import time
import matplotlib.pyplot as plt
import copy
import numpy as np

MAX_DISTANCE = 100

def random_symmetric_graph(n):
    ''' Symmetric adjacency matrix of size nxn '''
    dist_matrix = [[oo for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(i+1,n):
            v = randint(1,MAX_DISTANCE)
            dist_matrix[i][j] = v
            dist_matrix[j][i] = v
    return dist_matrix

def random_euclidean_graph(n):
    ''' Symmetric adjacency matrix of a Euclidean graph of size nxn '''
    dist_matrix = [[oo for _ in range(n)] for _ in range(n)]
    points = []
    for p in range(n):
        x,y = randint(0,MAX_DISTANCE), randint(0,MAX_DISTANCE)
        points.append((x,y))
    for i in range(n):
        p1 = points[i]
        for j in range(i+1,n):
            p2 = points[j]
            distance = sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)
            dist_matrix[i][j] = distance
            dist_matrix[j][i] = distance
    return dist_matrix

def show(G):
    ''' Show adjacency matrix. Useful for debugging. '''
    n = len(G)
    r = "     "
    for i in range(n):
        r += f'{i:4}'
    r += '\n    -'+'-'*(4*n)+'\n'
    for i in range(n):
        r += f'{i:2} | '
        for j in range(n):
            r += f'{G[i][j]:4}'
        r += '\n'
    r = r.replace('inf', '  ∞')
    print(r)

def cost(G, cycle):
    ''' Calculate the cost of the given cycle '''
    c = 0
    n = len(G)
    for i in range(n):
        a = cycle[i]
        b = cycle[(i+1)%n]
        c += G[a][b]
    return c

# Example

# Traveling Salesman Problem using Genetic Algorithm

Source:
* https://towardsdatascience.com/evolution-of-a-salesman-a-complete-genetic-algorithm-tutorial-for-python-6fe5d2b3ca35
*  https://www.geeksforgeeks.org/traveling-salesman-problem-using-genetic-algorithm/ (in C++)
* http://www.theprojectspot.com/tutorial-post/applying-a-genetic-algorithm-to-the-travelling-salesman-problem/5
* https://www.youtube.com/watch?v=XP8R0yzAbdo

# Create a first random generation of x possible routes

In [70]:
G = random_symmetric_graph(5)
population = 6

print(G)

show(G)

'''Used the shuffle function to generate a 'population' sized amount'''
'''of possible routes'''

def createGen(matrix, population):
    route = []
    genZ = [None] * int(population)
    for i in range(0,len(matrix)):
        route.append(i)
    for j in range(0,int(population)):
        if j == 0:
            np.random.shuffle(route)
            newRoute = route[:]
            genZ[j] = newRoute
        else:
            np.random.shuffle(route)
            for h in range(0,len(genZ)):
                if route == genZ[h]:
                    np.random.shuffle(route)
            newRoute = route[:]
            genZ[j] = newRoute
    return genZ

print("Genenration 0")        
print(createGen(G,6))

[[inf, 52, 12, 89, 53], [52, inf, 61, 94, 26], [12, 61, inf, 60, 26], [89, 94, 60, inf, 46], [53, 26, 26, 46, inf]]
        0   1   2   3   4
    ---------------------
 0 |    ∞  52  12  89  53
 1 |   52   ∞  61  94  26
 2 |   12  61   ∞  60  26
 3 |   89  94  60   ∞  46
 4 |   53  26  26  46   ∞

Genenration 0
[[1, 3, 2, 0, 4], [2, 4, 3, 0, 1], [2, 0, 3, 4, 1], [1, 2, 3, 0, 4], [0, 1, 3, 2, 4], [4, 2, 0, 3, 1]]


# Rank the "fitness" of these routes aka length

In [91]:
'''Turn routes into fitness values'''

def fitness(matrix, routes):
    fitnessRanking = []
    for i in range(0, len(routes)):
        fitnessTotal = 0
        for j in range(0,len(matrix)):
            if j + 1 < len(matrix):
                fitnessTotal = fitnessTotal + int(matrix[routes[i][j]][routes[i][j+1]])
            else:
                fitnessTotal = fitnessTotal + int(matrix[routes[i][j]][routes[i][0]])
        fitnessRanking.append(fitnessTotal)
    return fitnessRanking

show(G)
print("Genenration 0")        
genZ = createGen(G,6)
print(genZ)
print("Fitness")
ranking = fitness(G, genZ)
print(ranking)

        0   1   2   3   4
    ---------------------
 0 |    ∞  52  12  89  53
 1 |   52   ∞  61  94  26
 2 |   12  61   ∞  60  26
 3 |   89  94  60   ∞  46
 4 |   53  26  26  46   ∞

Genenration 0
[[1, 4, 0, 3, 2], [3, 4, 0, 2, 1], [0, 2, 3, 1, 4], [0, 3, 4, 2, 1], [3, 4, 2, 1, 0], [0, 4, 1, 2, 3]]
Fitness
[289, 266, 245, 274, 274, 289]


# Choose the parents for mating

Find two routes with the lowest value, shortest route

In [97]:
def first(matrix, ranking, routes):
    sortedRank = ranking[:]
    sortedRank.sort()
    firstPos = ranking.index(sortedRank[0])
    firstRoute = routes[firstPos]
    return firstRoute

def second(matrix, ranking, routes):
    sortedRank = ranking[:]
    sortedRank.sort()
    secondPos = ranking.index(sortedRank[1])
    secondRoute = routes[secondPos]
    return secondRoute
    
win = first(G, ranking, genZ)
print(win)
win2 = second(G, ranking, genZ)
print(win2)
    

[0, 2, 3, 1, 4]
[3, 4, 0, 2, 1]


# Breading

In [131]:
def breed(parent1, parent2, population):
    nextGen = []
    
    for i in range(0,population):
        child = []
        childP1 = []
        childP2 = []
    
    
        geneA = int(np.random.random() * len(parent1))
        geneB = int(np.random.random() * len(parent1))
    
        startGene = min(geneA, geneB)
        endGene = max(geneA, geneB)
    
        for i in range(startGene, endGene):
            childP1.append(parent1[i])
        
        childP2 = [item for item in parent2 if item not in childP1]
        child = childP1 + childP2
        nextGen.append(child)
    
    return nextGen
    
nextGen = breed(win, win2, population)
print(nextGen)

[[0, 3, 4, 2, 1], [2, 3, 4, 0, 1], [0, 2, 3, 4, 1], [0, 3, 4, 2, 1], [0, 2, 3, 4, 1], [2, 3, 4, 0, 1]]


# Mutation

Randomly mutate with a chosen rate

In [134]:
'''Mutation rate set to 5%'''

mutation = 0.05

def mutate(child, mutationRate):
    for swapped in range(len(child)):
        if(np.random.random() <= mutationRate):
            swapWith = int(np.random.random() * len(child))
            
            node1 = child[swapped]
            node2 = child[swapWith]
            
            child[swapped] = node2
            child[swapWith] = node1
    return child

def mutateGen(population, mutationRate):
    mutatedGen = []
    
    for child in range(0, len(population)):
        mutatedChild = mutate(population[child], mutationRate)
        mutatedGen.append(mutatedChild)
        
    return mutatedGen

mutatedGen = mutateGen(nextGen,mutation)
print(mutatedGen)

[[0, 3, 4, 2, 1], [2, 3, 0, 1, 4], [0, 2, 3, 4, 1], [0, 3, 4, 2, 1], [0, 2, 3, 4, 1], [2, 3, 4, 0, 1]]


# Repeat